In [1]:
%run StdPackages.ipynb

No clean-up of work-folder


Load wheels and test databases:

In [2]:
fs = [f"{d['data']}\\test_size1000.gdx", f"{d['data']}\\baselinerun.gdx"] # files
ws = gams.GamsWorkspace() 
g2np = gams2numpy.Gams2Numpy(ws.system_directory)
dbs = {'gms1': ws.add_database_from_gdx(fs[0]), 'gms2': ws.add_database_from_gdx(fs[1]),
       'gpy1': Database.GpyDB(db=fs[0],**{'name': 'testdb1'}), 'gpy2': Database.GpyDB(db=fs[1],**{'name': 'testdb2'})}
db = dbs['gpy2']

In [3]:
os.chdir(d['py'])
import DBWheels_robust, gmdcc
os.chdir(d['curr'])

# DBWheels_robust.py

The wheels ```DBWheels_robust.py``` defines more robust methods that work on different type of symbols and database types. Main methods are: ```robust_gpy, robust_add_or_merge, robust_merge_dbs```

### 1: ```robust_gpy```

```python
DBWheels_robust.robust_gpy(symbol, db = None, g2np = None, **kwargs):
```

The method initializes a ```gpy``` symbol. It takes a number of different input types:

```python
if isinstance(symbol, admissable_gpy_types):
```
    A gpy symbol is initialized using standard methods (see Class_gpy.ipynb). The **kwargs are only used in this instance.

```python
if isinstance(symbol,admissable_gamsTypes):
```
    A gpy symbol is initialized after reading relevant information using Database.gpydict_from_GamsSymbol


```python
else try:
    gpy(gpydict_from_GmdSymbol(db,g2np,symbol)):
```

    If the symbol belongs to a gams.GamsDatabase._gmd this will work. Otherwise we get an error.

##### 1.1 Initialize from pandas-like/python/gpy types

The following input types can be used (see Class_gpy.ipynb):

In [4]:
Database.admissable_gpy_types 

(pandas.core.series.Series,
 pandas.core.indexes.base.Index,
 int,
 float,
 str,
 numpy.generic,
 dict,
 _Database.gpy)

*Example:*

In [5]:
s = DBWheels_robust.robust_gpy(pd.Series([0], index = pd.Index(['i1'], name = 'i'), name = 'new_variable'))
s.__dict__

{'vals': i
 i1    0
 Name: new_variable, dtype: int64,
 'name': 'new_variable',
 'type': 'variable',
 'text': ''}

##### 1.2 Initialize gams type:

*Takes gams._GamsSymbol, gams.GamsDatabase, and gams2numpy input:*

In [6]:
s = DBWheels_robust.robust_gpy(db.database['k'], db=db.database, g2np = db.g2np)
s.__dict__

{'vals': Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
             18, 19, 20, 21, 22, 23, 24, 25],
            dtype='int64', name='k'),
 'name': 'k',
 'text': 'short run states of the model. Can at most be 8760 as h_full',
 'type': 'set'}

##### 1.3 Initialize from '_gmd' type:

*This takes a gmd-like symbol, a gams.GamsDatabase._gmd database, and gams2numpy input:*

In [7]:
db_gmd,rc = db.database._gmd, gmdcc.new_intp()
gmd_symbol = gmdcc.gmdGetSymbolByNumberPy(db_gmd, 1, rc)
s = DBWheels_robust.robust_gpy(gmd_symbol, db = db_gmd, g2np = db.g2np)
s.__dict__

{'vals': Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
             18, 19, 20, 21, 22, 23, 24, 25],
            dtype='int64', name='k'),
 'name': 'k',
 'text': 'short run states of the model. Can at most be 8760 as h_full',
 'type': 'set'}

### 2: ```robust_add_or_merge```

```python
DBWheels_robust.robust_add_or_merge(db, symbol, db_from = None, g2np = None, merge=False, **kwargs):
```

The method initializes a ```gpy``` symbol ```s``` using ```robust_gpy(symbol,db=db_from, g2np = g2np, **kwargs)```. This is either added or merged to the database ```db```.

```python
if isinstance(db, (dict, GpyDB, SeriesDB)):
```
    The db consists of (s.name, s) items. If s.name is already in the database merge values. If not, add the new symbol.

```python
if isinstance(symbol,gams.GamsDatabase):
```
     The symbol either added or merged ultimately using the gams2numpy method ```gmdFillSymbolStr```.

*Example: dictionary, GpyDB, SeriesDB or gams.Gamsdatabse types:*

In [8]:
dbs = [{}, db, db.series, db.database]

*Add a gpy to each of the different types*

In [9]:
for db_i in dbs:
    DBWheels_robust.robust_add_or_merge(db_i, s, db_from = db.database, g2np = db.g2np)
    print(f"Symbol added to db type {type(db_i)}, \n\tas type: {db_i[s.name]}")

Symbol added to db type <class 'dict'>, 
	as type: <_Database.gpy object at 0x000001A46101B6D0>
Symbol added to db type <class 'Database.GpyDB'>, 
	as type: <_Database.gpy object at 0x000001A465C43400>
Symbol added to db type <class 'Database.SeriesDB'>, 
	as type: <_Database.gpy object at 0x000001A465C43400>
Symbol added to db type <class 'gams.database.GamsDatabase'>, 
	as type: <gams.database.GamsSet object at 0x000001A4660C0FD0>


### 3: ```robust_merge_dbs```

```python
DBWheels_robust.robust_merge_dbs(db1, db2, priority='second'):
```

The method merges all symbols in ```db2``` into ```db1```. If symbols are in both databases, the priority will be given to ```priority```. Note that, generally, the method ```first``` will be slower. The two databases, ```(db1,db2)``` can both be of types ```(GpyDB,SeriesDB,dict,gams.GamsDatabase)```. 

*The following tests (trivial) merging of the different types:*

In [10]:
db_combinations = [(k,v) for k in dbs for v in dbs]
for (db1,db2) in db_combinations:
    DBWheels_robust.robust_merge_dbs(db1,db2)

*Test reversing the priority:*

In [11]:
for (db1,db2) in db_combinations:
    DBWheels_robust.robust_merge_dbs(db1,db2,priority = 'first')